# Product recommendation using a Retail Dataset

In this notebook, we'll demostrate how to find product recommedations based on previous purchase history

## Objectives

* Explore the dataset
* Clean the dataset
* Provide non-personalized recommendations
* Create user-item interaction matrix 
* Personalized recommendation using user-based collaborative filtering
* Personalized recommendation using item-based collaborative filtering

## The problem:
Suppose that we are working in an e-commerce giant named Globomantics. We have a website which acts as an online store where customers can visit and purchase products. We also keep a record of all the past purchases in our databases. 

Now, we want to build a product recommendations system which will suggest products to our customers. We want to be sure that the products are relevant to the customers so that they'll be lured into buying those products increasing the overall revenue.

## The solution:
### Non-personalized Recommendation
We can provide a list of popular products to the customers.

### Personalized Recommendations
We can provide customized product recommendations based on customer's past purchase history.

## Dataset

The dataset which we'll be using throughout this exercise is a public dataset available in the UCI machine learning Repository. More information about the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail) and it can be downloaded also from the same link. 

This is a transactional dataset which contains all the transactions occurring between Dec 01, 2010 and Dec 09, 2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.
 
#### Attribute Information:
* InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
* StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
* Description: Product (item) name. Nominal. 
* Quantity: The quantities of each product (item) per transaction. Numeric. 
* InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated. 
* UnitPrice: Unit price. Numeric, Product price per unit in sterling. 
* CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. 
* Country: Country name. Nominal, the name of the country where each customer resides.

## Explore the dataset

In [1]:
# import Pandas
import pandas as pd

In [2]:
# The retail data is downloaded and saved in a CSV file
# Read the CSV file and create a Pandas dataframe
retail_data = pd.read_csv('RetailData.csv')

In [3]:
# Check the total number of rows and columns of the dataframe
print(retail_data.shape)

(541909, 8)


In [4]:
print(list(retail_data.columns))

['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [5]:
# Check the first few rows of the dataframe
retail_data.head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


## Clean the dataset

We'll now remove rows containing null values. You can learn more about dropna method [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) 

In [6]:
retail_data.dropna(inplace=True)

As you have noticed, the customer ids are read as floting point numbers. But we need them as strings. The following code will do the trick. To know more about "apply" method and lamdba function, go through the following links - [link1](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html), [link2](https://www.w3schools.com/python/python_lambda.asp), [link3](https://towardsdatascience.com/lambda-functions-with-practical-examples-in-python-45934f3653a8)

In [7]:
# Prepend 'C' to CustomerID to make it a string
retail_data['CustomerID'] = retail_data['CustomerID'].apply(lambda x: 'C'+str(int(x)))
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,C17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,C17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,C17850,United Kingdom


In [8]:
# DataFrame for building the recommendation system
orders = retail_data[['CustomerID', 'Description', 'InvoiceNo', 'Quantity']]
orders.head()

,CustomerID,Description,InvoiceNo,Quantity
0,C17850,WHITE HANGING HEART T-LIGHT HOLDER,536365,6
1,C17850,WHITE METAL LANTERN,536365,6
2,C17850,CREAM CUPID HEARTS COAT HANGER,536365,8
3,C17850,KNITTED UNION FLAG HOT WATER BOTTLE,536365,6
4,C17850,RED WOOLLY HOTTIE WHITE HEART.,536365,6


In [9]:
# Drop descriptions that are not uppercase
orders = orders[
    orders['Description'].str.upper() == orders['Description']
]
orders.head()

,CustomerID,Description,InvoiceNo,Quantity
0,C17850,WHITE HANGING HEART T-LIGHT HOLDER,536365,6
1,C17850,WHITE METAL LANTERN,536365,6
2,C17850,CREAM CUPID HEARTS COAT HANGER,536365,8
3,C17850,KNITTED UNION FLAG HOT WATER BOTTLE,536365,6
4,C17850,RED WOOLLY HOTTIE WHITE HEART.,536365,6


## Non-personalized recommendation

Panda's "groupby" is used to create groups withing the data. Here, we'll group by the product description which will create a group for each product. Inside each group, there will be lots of orders identified by invoice number. We can use the "agg" method to count the number of unique "InvoiceNo" inside each group. This we rename as orders. So, we get a list of product description and number of orders pair. We sort this by descending order of number of orders and output the first 10. 

You can learn more about [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html), [agg](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html), [sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) from the official Panda's documentation.

In [10]:
orders.groupby('Description').agg(
    orders=('InvoiceNo', 'nunique')
)

,orders
Description,
4 PURPLE FLOCK DINNER CANDLES,36
50'S CHRISTMAS GIFT BAG LARGE,110
DOLLY GIRL BEAKER,135
I LOVE LONDON MINI BACKPACK,68
I LOVE LONDON MINI RUCKSACK,1
...,...
ZINC T-LIGHT HOLDER STARS SMALL,240
ZINC TOP 2 DOOR WOODEN SHELF,11
ZINC WILLIE WINKIE CANDLE STICK,189


In [11]:
orders.groupby('Description').agg(
    orders=('InvoiceNo', 'nunique'),
).sort_values(by='orders', ascending=False).head(10)

,orders
Description,
WHITE HANGING HEART T-LIGHT HOLDER,2013
REGENCY CAKESTAND 3 TIER,1884
JUMBO BAG RED RETROSPOT,1643
PARTY BUNTING,1399
ASSORTED COLOUR BIRD ORNAMENT,1385
LUNCH BAG RED RETROSPOT,1329
SET OF 3 CAKE TINS PANTRY DESIGN,1218
POSTAGE,1194
LUNCH BAG BLACK SKULL.,1073


In [12]:
orders.groupby('Description').agg(
    quantity=('Quantity', 'sum')
).sort_values(by='quantity', ascending=False).head(10)

,quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409
POPCORN HOLDER,30504
RABBIT NIGHT LIGHT,27094
MINI PAINT SET VINTAGE,25880
PACK OF 12 LONDON TISSUES,25321


## Create user-item interaction matrix

We'll now convert our original order dataframe into a matrix whose rows corresponds to the customers and columns correspond to the items. This can be achieved by using the Panda's "pivot_table" function. We'll have to specify that we want customers as our rows, the product description as our columns and the quantity as the value inside the matrix.

You can read more about [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) from the official documentation.

In [13]:
int_matrix = orders.pivot_table(index='CustomerID', columns=['Description'], values='Quantity').fillna(0)
int_matrix.head(5)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
C12346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## User-based collaborative filtering

We'll first select a user as our active user. Then, we'll calculate the similarity of this active user with all the other users. We'll use correlation as our similarity measure here. We can use Pandas's built-in function "corrwith" to calculate the correlations. Some users may not be correlated at all and produce null values which we'll need to remove. We'll then create a dataframe out of the correlation values with customer id as one of the columns and correlation as the other. We'll sort this dataframe in descending order and output the top 10 most similar users to the active user.

Read more about [corrwith](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corrwith.html) in official Panda's documentation.

In [14]:
active_user = 'C12347'

In [15]:
recommendations = int_matrix.corrwith(int_matrix.loc[active_user], axis=1)
recommendations.dropna(inplace=True)
recommendations

CustomerID
C12347    1.000000
C12348    0.163545
C12349    0.005202
C12350    0.015139
C12352    0.004437
            ...   
C18280    0.007461
C18281   -0.003102
C18282    0.010109
C18283    0.019187
C18287    0.101071
Length: 4350, dtype: float64

In [16]:
recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
recommendations = recommendations.sort_values(by='correlation', ascending=False)
recommendations.head(10)

,CustomerID,correlation
0,C12347,1.000000
1455,C14326,0.779165
7,C12355,0.710882
1410,C14257,0.620565
889,C13532,0.572588
2607,C15882,0.521028
2350,C15539,0.520732
2243,C15396,0.520150
2107,C15215,0.515967
4043,C17866,0.504141


In [17]:
active_user = 'C12347'
similar_user = 'C14326'

for item, quantity in int_matrix.loc[similar_user].items():
    if quantity > 0 and int_matrix.loc[active_user][item] == 0:
        print("Name of the item: ", item)
        print("Quantity purchased by similar user: ", quantity)
        print("Quantity purchased by active user: ", int_matrix.loc['C12347'][item])

Name of the item:  ANTIQUE SILVER T-LIGHT GLASS
Quantity purchased by similar user:  12.0
Quantity purchased by active user:  0.0
Name of the item:  ENAMEL FLOWER JUG CREAM
Quantity purchased by similar user:  3.0
Quantity purchased by active user:  0.0
Name of the item:  FELTCRAFT 6 FLOWER FRIENDS
Quantity purchased by similar user:  12.0
Quantity purchased by active user:  0.0
Name of the item:  GROW YOUR OWN BASIL IN ENAMEL MUG
Quantity purchased by similar user:  8.0
Quantity purchased by active user:  0.0
Name of the item:  HANGING HEART JAR T-LIGHT HOLDER
Quantity purchased by similar user:  12.0
Quantity purchased by active user:  0.0
Name of the item:  SET/3 OCEAN SCENT CANDLE JEWEL BOX
Quantity purchased by similar user:  4.0
Quantity purchased by active user:  0.0


## Item-based collaborative filtering

In [18]:
item = 'WHITE HANGING HEART T-LIGHT HOLDER'

In [19]:
recommendations = int_matrix.corrwith(int_matrix[item])
recommendations.dropna(inplace=True)
recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
recommendations = recommendations.sort_values(by='correlation', ascending=False)
recommendations.head(10)

,Description,correlation
3680,WHITE HANGING HEART T-LIGHT HOLDER,1.000000
1378,GIN + TONIC DIET METAL SIGN,0.704755
3404,TEA TIME TEA TOWELS,0.650505
1153,FAIRY CAKE FLANNEL ASSORTED COLOUR,0.593688
2679,RED HANGING HEART T-LIGHT HOLDER,0.530743
1006,DOORMAT FAIRY CAKE,0.515547
2177,OPEN CLOSED METAL SIGN,0.476353
2784,RETROSPOT HEART HOT WATER BOTTLE,0.476083
2388,PIGGY BANK RETROSPOT,0.476052
333,BLACK AND WHITE CAT BOWL,0.473697
